In [10]:
from bs4 import BeautifulSoup

with open("Energy.htm") as fp:
    soup = BeautifulSoup(fp, "lxml")

In [11]:
#print(soup.prettify())

In [12]:
topmovers = soup.find_all('table', class_='topmovers')
#print(topmovers.prettify())
#type(topmovers)
trs = topmovers[0].find_all('tr')
trs[1]
equity = trs[1].find('a').get_text()
change = trs[1].find('span', class_="chg").get_text()
changes = trs[1].find_all('span', class_="chg")
change = changes[1].get_text()
change = change[1:][:-2]
change

'5.31'

In [13]:
len(topmovers[0].find_all('tr'))
tr = topmovers[0].find('tr')     

In [14]:
# topmover = soup.find('table', class_='topmovers')
# print(topmover.prettify())
#equity = topmovers[0].find_all('tr').find('a').get_text()
#equity
# change = topmover.find('span', class_="chg").get_text()
# change

In [15]:
with open("Google finance.htm") as fp:
    soup = BeautifulSoup(fp, "lxml")

In [16]:
sector = soup.find_all('div', id='secperf')
trs = sector[0].find_all('tr')

#trs[7].find('span', class_="chr").get_text()[:-1]
#trs[7].find('a').get_text()

sum = {}
for i in range(1,8,3):
    sector = trs[i].find('a').get_text()
    change = trs[i].find('span', class_="chr").get_text()[:-1]
    sum[sector] = float(change)
sum

{'Basic Materials': -0.35, 'Energy': -1.48, 'Industrials': -0.46}

In [17]:
def get_sector_sum():
    with open("Google Finance.htm") as fp:
        soup = BeautifulSoup(fp, "lxml")
        
    sector = soup.find_all('div', id='secperf')
    trs = sector[0].find_all('tr')
    sum = {}
    
    for i in range(1,8,3):
        sector = trs[i].find('a').get_text()
        change = trs[i].find('span', class_="chr").get_text()[:-1]
        sum[sector] = float(change)
        
    return sum

In [18]:
def get_movers(file):
    movers = {}
    
    with open(file) as fp:
        soup = BeautifulSoup(fp, "lxml")

    try:
        topmovers = soup.find_all('table', class_='topmovers')
        trs = topmovers[0].find_all('tr')
        for tr in trs[1:6]:
            equity = tr.find('a').get_text()
            change = tr.find_all('span', class_="chg")[1].get_text()[1:][:-2]
            movers[equity] = float(change)
            
        for tr in trs[7:12]:
            equity = tr.find('a').get_text()
            change = tr.find_all('span', class_="chr")[1].get_text()[1:][:-2]
            movers[equity] = float(change)
            
        return movers
    except:
        return None
    

In [19]:
def get_top_equity(movers, biggest):
    for equity in movers:
        if movers[equity] == biggest:
            break
            
    return equity 


In [20]:
#len(get_changes("Energy.htm"))
#Energy.items()

def get_biggest(sector):
    biggest_gainer = {}
    biggest_loser = {}
    
    file = sector + ".htm"
    movers = get_movers(file)
    gainer_c = max(movers.values())
    loser_c = min(movers.values())

    biggest_gainer["equity"] = get_top_equity(movers, gainer_c)
    biggest_gainer["change"] = gainer_c

    biggest_loser["equity"] = get_top_equity(movers, loser_c)
    biggest_loser["change"] = loser_c
    
    return [biggest_gainer, biggest_loser]


In [21]:
def google_sector_report():
    from bs4 import BeautifulSoup
    import json
    
    results = {}
    result = {}
    Energy = {}
    BasicMaterials = {}
    Industrials = {}

    Energy["biggest_gainer"] = get_biggest("Energy")[0]
    Energy["biggest_loser"] = get_biggest("Energy")[1]
    Energy["change"] = get_sector_sum()["Energy"]
    BasicMaterials["biggest_gainer"] = get_biggest("Basic Materials")[0]
    BasicMaterials["biggest_loser"] = get_biggest("Basic Materials")[1]
    BasicMaterials["change"] = get_sector_sum()["Basic Materials"]
    Industrials["biggest_gainer"] = get_biggest("Industrials")[0]
    Industrials["biggest_loser"] = get_biggest("Industrials")[1]
    Industrials["change"] = get_sector_sum()["Industrials"]

    result["Energy"] = Energy
    result["Basic Materials"] = BasicMaterials
    result["Industrials"] = Industrials
    results["result"] = result
    
    return json.dumps(results)

In [24]:
google_sector_report()

'{"result": {"Energy": {"biggest_gainer": {"equity": "McDermott International", "change": 5.31}, "biggest_loser": {"equity": "Bill Barrett Corporation", "change": -13.18}, "change": -1.48}, "Basic Materials": {"biggest_gainer": {"equity": "Gold Fields Limited (ADR)", "change": 3.22}, "biggest_loser": {"equity": "Jaguar Mining Inc (USA)", "change": -5.85}, "change": -0.35}, "Industrials": {"biggest_gainer": {"equity": "LML Payment Systems, Inc.", "change": 633.43}, "biggest_loser": {"equity": "Chicago Bridge & Iron Co", "change": -12.07}, "change": -0.46}}}'